In [2]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')

In [3]:
외국인역대성적 = pd.read_csv('kbo_yearly_foreigners_2011_2018.csv')
외국인메이저성적 = pd.read_csv('fangraphs_foreigners_2011_2018.csv')
외국인스탯캐스터 = pd.read_csv('baseball_savant_foreigners_2011_2018.csv')
신규외국인성적 = pd.read_csv('fangraphs_foreigners_2019.csv')
신규외국인스텟캐스터 = pd.read_csv('baseball_savant_foreigners_2019.csv')

df = pd.read_csv('최종외국인스탯캐스터.csv',encoding='euc-kr',engine="python")

In [4]:
throw_df = pd.read_csv('plus_win_lose_2.csv')
throw_df = throw_df.fillna(0)
print(throw_df.shape)
top20 = throw_df.sort_values(['승률','경기수'], ascending=False).reset_index(drop=True)[:20]
# bottom20 = throw_df.sort_values(['승률','경기수'], ascending=False).reset_index(drop=True)[-20:]
# display(top20)
# display(bottom20)

(105, 16)


In [5]:
throw_df.drop('year',axis=1,inplace=True)

### 머신러닝

In [6]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [7]:
throw_df['year_born'] = throw_df['year_born'].fillna(0)
throw_df['방출연도'] = throw_df['방출연도'].fillna(0)

In [18]:
throw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 16 columns):
pitcher_name    105 non-null object
year            105 non-null int64
team            105 non-null object
ERA             105 non-null float64
TBF             105 non-null int64
H               105 non-null int64
HR              105 non-null int64
BB              105 non-null int64
HBP             105 non-null int64
SO              105 non-null int64
year_born       105 non-null object
승               105 non-null int64
패               105 non-null int64
승률              105 non-null float64
경기수             105 non-null int64
방출연도            105 non-null float64
dtypes: float64(3), int64(10), object(3)
memory usage: 13.2+ KB


In [9]:
from sklearn.preprocessing import LabelEncoder

def label(df):
    col = list(df)
    encoder = LabelEncoder()
    encoder.fit(col)
    col_label = encoder.transform(col)
    return col_label

pitcher_name_label = label(throw_df['pitcher_name'])
team_label = label(throw_df['team'])
year_born_label = label(throw_df['year_born'])

In [25]:
throw_df = throw_df.assign(pitcher_name_label=pitcher_name_label,
                          team_label=team_label,
                          year_born_label=year_born_label)
throw_df_digit = throw_df.drop(['pitcher_name','team','year_born','year'],axis=1)
throw_df_digit.head()

KeyError: "labels ['year'] not contained in axis"

In [26]:
feature = throw_df_digit.drop('승',axis=1)
label = throw_df_digit['승']

In [27]:
X_train, X_test, y_train, y_test = train_test_split(feature,label, test_size = 0.2, random_state=20)

In [28]:
base_dtc = DecisionTreeClassifier(random_state=20, criterion='entropy')

In [29]:
base_dtc.fit(X_train, y_train)
predition = base_dtc.predict(X_test)

In [30]:
print(' y_test\n', y_test)
print(' prediction\n', predition)

 y_test
 72     12
78     10
31     19
5      14
2      11
46      4
38     13
100     9
93      7
0      11
39      7
60      7
81      5
12      7
98      6
36     10
48     18
23      5
65      2
53     10
92      9
Name: 승, dtype: int64
 prediction
 [11  7 15 10 10  4 14 13  6 10  6  4  5  8  8 10 15  5  3  8  8]


In [31]:
from sklearn.metrics import accuracy_score
print(f'예측 정확도 : {accuracy_score(y_test, predition)}')

예측 정확도 : 0.19047619047619047


### 투수 - 외국 경기 평균 변환

#### 외국인메이저성적에 평균 구속과 구종 개수 추가

In [14]:
외국인메이저성적.head()

,pitcher_name,year,ERA,WAR,TBF,H,HR,BB,HBP,SO,WHIP,BABIP,FIP,LD%,GB%,FB%,IFFB%,SwStr%,Swing%
0,오간도,2011.0,3.51,3.3,693.0,149.0,16.0,43.0,7.0,126.0,1.14,0.265,3.65,0.237,0.364,0.674,0.147,0.090,0.475
1,험버,2011.0,3.75,3.2,676.0,151.0,14.0,41.0,6.0,116.0,1.18,0.275,3.58,0.168,0.471,0.458,0.094,0.092,0.463
2,루카스,2012.0,3.76,2.8,827.0,185.0,13.0,78.0,1.0,140.0,1.36,0.289,3.75,0.203,0.572,0.707,0.082,0.062,0.424
3,다이아몬드,2012.0,3.54,2.2,714.0,184.0,17.0,31.0,4.0,90.0,1.24,0.292,3.94,0.210,0.534,0.597,0.040,0.068,0.467
4,듀브론트,2013.0,4.32,2.2,705.0,161.0,13.0,71.0,5.0,139.0,1.43,0.310,3.78,0.199,0.456,0.633,0.127,0.077,0.434


In [5]:
평균구속 = df.groupby('pitcher_name',as_index=False).agg({'km변환':'mean'})\
.sort_values('km변환',ascending=False).reset_index(drop=True)

In [6]:
for idx, value in 평균구속.iterrows():
    for i in range(외국인메이저성적.shape[0]):
        if 외국인메이저성적.loc[i,'pitcher_name'] == value[0]:
            외국인메이저성적.loc[i,'평균구속'] = value[1]

In [7]:
외국인메이저성적[외국인메이저성적['평균구속'].isna() == True]

,pitcher_name,year,ERA,WAR,TBF,H,HR,BB,HBP,SO,WHIP,BABIP,FIP,LD%,GB%,FB%,IFFB%,SwStr%,Swing%,평균구속
67,벨레스터,2010.0,2.57,0.1,89.0,15.0,2.0,11.0,2.0,28.0,1.24,0.283,3.51,0.114,0.568,0.622,0.071,0.145,0.420,NaN
106,카스티요,2017.0,13.50,0.0,8.0,3.0,0.0,1.0,0.0,2.0,3.00,0.600,2.41,0.600,0.400,0.500,0.000,0.200,0.467,NaN
139,리즈,2015.0,4.24,-0.2,106.0,26.0,4.0,12.0,3.0,27.0,1.63,0.367,4.98,0.349,0.349,0.690,0.158,0.103,0.461,NaN
171,벨레스터,2015.0,7.47,-0.3,77.0,17.0,3.0,13.0,1.0,13.0,1.91,0.298,6.64,0.160,0.280,0.718,0.071,0.104,0.429,NaN
180,벨레스터,2011.0,4.54,-0.4,159.0,38.0,7.0,14.0,1.0,34.0,1.46,0.301,4.93,0.194,0.417,0.691,0.048,0.072,0.427,NaN
190,벨레스터,2012.0,6.50,-0.5,83.0,14.0,5.0,11.0,3.0,12.0,1.39,0.173,7.71,0.161,0.321,0.684,0.069,0.077,0.418,NaN


In [30]:
for val in 외국인메이저성적[외국인메이저성적['평균구속'].isna() == True].drop_duplicates(['pitcher_name'])['pitcher_name']:
    display(df[df['pitcher_name'] == val])

,game_date,release_speed,batter,pitcher,events,description,zone,stand,p_throws,bb_type,...,ax,ay,az,launch_speed,launch_angle,release_spin_rate,pitch_name,pitcher_name,km변환,type


,game_date,release_speed,batter,pitcher,events,description,zone,stand,p_throws,bb_type,...,ax,ay,az,launch_speed,launch_angle,release_spin_rate,pitch_name,pitcher_name,km변환,type


,game_date,release_speed,batter,pitcher,events,description,zone,stand,p_throws,bb_type,...,ax,ay,az,launch_speed,launch_angle,release_spin_rate,pitch_name,pitcher_name,km변환,type


In [7]:
외국인메이저성적_not_na = 외국인메이저성적[외국인메이저성적['평균구속'].isna() == False].reset_index(drop=True)

외국인메이저성적_drop_year = 외국인메이저성적.drop('year', axis=1)

외국인메이저_평균 = 외국인메이저성적_drop_year.groupby('pitcher_name').agg('mean').head().reset_index(drop=False).round(decimals=2)

In [26]:
### 구종 개수 추가
playername = list(throw_df['pitcher_name'].drop_duplicates())

구종데이터 = pd.DataFrame({'pitcher_name':[], '2-Seam Fastball':[],'4-Seam Fastball':[],'Changeup':[],'Curveball':[],
                        'Cutter':[],'Intentional Ball':[],'Pitch Out':[],'Slider':[]})
for name in playername:
    try:
        a = 외국인스탯캐스터.groupby(['pitcher_name','pitch_name']).agg({'pitch_name':'count'})\
    .T[name].reset_index(drop=True)
        a['pitcher_name'],a['cnt'] = name, a.shape[1]
        구종데이터 = 구종데이터.append(a,ignore_index=True)
    except:
        print(name)
display(구종데이터.head())
print(구종데이터.cnt.mean())

,2-Seam Fastball,4-Seam Fastball,Changeup,Curveball,Cutter,Eephus,Fastball,Forkball,Intentional Ball,Pitch Out,Sinker,Slider,Split Finger,Unknown,cnt,pitcher_name
0,26.0,384.0,81.0,82.0,6.0,NaN,NaN,NaN,4.0,1.0,NaN,4.0,NaN,NaN,8.0,니퍼트
1,19.0,245.0,8.0,NaN,20.0,NaN,NaN,NaN,4.0,2.0,269.0,306.0,NaN,NaN,8.0,소사
2,58.0,433.0,429.0,NaN,171.0,NaN,NaN,NaN,9.0,5.0,1360.0,284.0,NaN,NaN,8.0,탈보트
3,773.0,950.0,297.0,88.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,256.0,NaN,NaN,6.0,레이예스
4,207.0,324.0,182.0,25.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,153.0,NaN,NaN,6.0,세든


6.3
